## Topics Categorization with LDA

## reference: [How we Changed Unsupervised LDA to Semi-Supervised GuidedLDA](https://www.freecodecamp.org/news/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164/)
### code：[GuidedLDA: Guided Topic modeling with latent Dirichlet allocation](https://github.com/vi3k6i5/GuidedLDA)

In [1]:
# test data
import pandas as pd
df = pd.read_csv('./WF Cases 20190517.csv')
df.head(10)

,Case Number,Subject,Description,Product Name,Product Version,Date/Time Opened,Solution Provided
0,00677507,[WFBS-SVC] Installation on Mac Is Not Full Fea...,NaN,Worry-Free Business Security Services,NaN,1/1/2018 12:18 AM,NaN
1,00677513,[WFBS - A] Smart Scan is not updating,CRC proc crash instability unable to keep syst...,Worry-Free Business Security Advanced,9.5,1/1/2018 2:30 AM,NaN
2,00677518,[WFBS - SVC] Installation issue,"Migrated a client to a new computer, the softw...",Worry-Free Business Security Services,6.2 (Beta),1/1/2018 6:41 AM,NaN
3,00677669,[MALWARE][WFBS SVC] Wallet Ransomware,NaN,Worry-Free Business Security Services,NaN,1/1/2018 11:54 PM,NaN
4,00677682,[MALWARE][WFBS S 9.5]Possible Ransomware detec...,OMAN wfbs SYED AIJAZ LOAY INTERNATIONAL LLC ai...,Worry-Free Business Security Standard,9.5,1/2/2018 12:30 AM,Next Action Plan: - check detection on the con...
5,00677799,[WFBS-S] Adding Exclusion to Behavior Monitoring,[Case Description] - client SWA financial plan...,Worry-Free Business Security Standard,NaN,1/2/2018 10:37 AM,NaN
6,00677803,[WFBS-S] Migrating from An Old Server to A New...,Issue: - customer wants to migrate WFBS to new...,Worry-Free Business Security Standard,NaN,1/2/2018 10:49 AM,NaN
7,00677816,[WFBS-SVC] Agent cloning,Can Worry Free agent with OS be image cloned/g...,Worry-Free Business Security Services,6.2,1/2/2018 12:02 PM,- manually reset the GUID info on the registry
8,00677831,[WFBS-SVC] Main Software not Functioning with ...,The latest Trend Micro update prevents one of ...,Worry-Free Business Security Services,6.2 (Beta),1/2/2018 1:13 PM,NaN
9,00677857,[WFBS-SVC] False Positive: Application Detecte...,[Issue]: False Positive: Application Detected ...,Worry-Free Business Security Standard,NaN,1/2/2018 2:38 PM,NaN


In [2]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from nltk.stem.porter import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer

import krovetz
from nltk.tokenize import word_tokenize

stop_words=stopwords.words('english')
break_words=list('\n\t')
prefix_char_remove=list('.*,-')

stemmer = krovetz.PyKrovetzStemmer()
#stemmer = SnowballStemmer("english")
#stemmer.stem('working')
#lemmatizer = WordNetLemmatizer()
word_freqs = dict()


## 以 regular expression 去除 email、數字、URL、Phone No、序號

In [34]:
re_pattern_list=[]
re_pattern_list.append('[a-zA-Z0-9_]+\.(com|org|net)[a-zA-Z0-9_\.]*$') #email
re_pattern_list.append('[0-9_]+') # all digits
re_pattern_list.append("\/\/[\w.-]+(?:\.[\w\.-]+)+[\w\-\._~:/?#[\]@!\$&'\(\)\*\+,;=.]+$") # url
re_pattern_list.append('[0-9\-]+') # phone no
re_pattern_list.append('[\u0800-\u4e00]+') # 日文
re_pattern_list.append('(trend|trend\smicro)') # trend micro


## 字詞清理函數

In [35]:
# clean words
def clean_word(word_0):
    word_list=[]
    for word in word_0.split('/'):

        if len(word) <= 0 :
            continue

        # remove 網路芳鄰路徑
        remote_path=word.split('\\')
        if len(remote_path) > 0 and len(remote_path[-1])>0:
            word=remote_path[-1]
        elif len(remote_path) > 1 and len(remote_path[-2])>0:
            word=remote_path[-2]
        elif len(remote_path) > 2 and len(remote_path[-3])>0:
            word=remote_path[-3]

        if word[0] == '"' and word[-1] == '"':
            word=word[1:-1]
        if word[0] == "'" and word[-1] == "'":
            word=word[1:-1]
        if len(word) <= 0 :
            continue

        # repeat 3 times to remove continous chars
        if word[0] in prefix_char_remove:
            word=word[1:]
        if len(word) <= 0 :
            continue
        if word[0] in prefix_char_remove:
            word=word[1:]
        if len(word) <= 0 :
            continue
        if word[0] in prefix_char_remove:
            word=word[1:]

        # lemmatize
        word=word.strip().lower()
        word=stemmer.stem(word)
        if word=='':
            continue      
            
        # remove .exe and .com
        if word.endswith('.exe') or word.endswith('.com'):
            word=word[:-4]
            
        # 以 regular expression 去除 email、數字、URL、Phone No、序號
        is_re_list = False
        for pattern1 in re_pattern_list:
            result=re.findall(pattern1, word)
            if  len(result) > 0:
                is_re_list = True
                break
        if is_re_list:
            break
            
            
        # remove the words with one or two characters only
        if len(word) <= 2 :
            continue
        if not (word in stop_words):
            word_list.append(word)
               
    return word_list

## test get service_list

In [36]:
df.Subject[0]

'[WFBS-SVC] Installation on Mac Is Not Full Featured'

In [37]:
import re
pattern1 = '\[[a-zA-Z0-9_\-]+\]' # category
re.findall(pattern1, '[WFBS-SVCaaa]'+df.Subject[0])

['[WFBS-SVCaaa]', '[WFBS-SVC]']

## 分詞

In [38]:
import re
pattern1 = '\[[a-zA-Z0-9_\-\s]+\]' # service_list pattern, e.g. [WFBS-SVC]

# 清理後的 email DESCRIPTION
clean_corpus=[]
original_corpus=[]
for index, line in df.iterrows():
    clean_line=""
    line = line['Subject']
    
    # remove service_list, e.g. [WFBS-SVC]
    if type(line) == str and len(line) > 0: 
        service_list = re.findall(pattern1, line)
        for service_item in service_list:
            line = line.replace(service_item, ' ')          
                                  
    #print(line)
    for break_word in break_words:
        #print('-',len(break_word), break_word, '-')
        if not type(line) == str or len(line) <= 0: 
            break
        line = line.replace(break_word, ' ')
    if not type(line) == str or len(line) <= 0: 
        #print(type(line))
        continue
    words = word_tokenize(line) #line.lower().split(' ')
    for word_0 in words:        
        word_list = clean_word(word_0)
        for word in word_list:        
            if word in word_freqs:
                word_freqs[word] += 1
            else:
                word_freqs[word] = 1
            clean_line+=' '+word
    original_corpus.append(line)
    clean_corpus.append(clean_line.strip())
email_words=word_freqs.keys()            
print(len(email_words))

4660


In [39]:
import re
keyword_list_new=[]
keyword_list = set(email_words)
for item in keyword_list:
    is_re_list = False
    for pattern1 in re_pattern_list:
        result=re.findall(pattern1, item)
        if  len(result) > 0:
            is_re_list = True
            break
    if is_re_list == False:
        keyword_list_new.append(item)
len(keyword_list_new)

4645

## 找出無母音的關鍵字

In [40]:
keyword_list_NoVowel={}
i=0
for word in keyword_list_new:
    if not set('aeiou').intersection(word) and len(word.replace('=','')) > 0 and len(word.replace('+','')) > 0 :
        if word in keyword_list_NoVowel.keys():
            keyword_list_NoVowel[word]+=1
        else:
            keyword_list_NoVowel[word]=1
#print(keyword_list_NoVowel.keys())   

In [41]:
import pickle
with open("keyword_list_NoVowel_2.pickle", 'wb') as f:
    pickle.dump(keyword_list_NoVowel, f)

## Load List of Microsoft software and Ubuntu Glossaries

In [42]:
import pickle
with open("keyword_list.pickle", 'rb') as f:
    keyword_list_TERM = pickle.load(f)


## compare corpus with n-gram keywords, n=2~4

In [43]:
def check_special_term(words, start_index, n_gram):
    merge_word=[]
    for i in range(n_gram):
        merge_word.append(words[start_index+i])
    if ' '.join(merge_word) in keyword_list_TERM:
        #print('converted_word=', keyword_list_TERM[' '.join(merge_word)]['converted_word'])
        return keyword_list_TERM[' '.join(merge_word)]['converted_word']
    else:
        return ''

def compare_corpus_with_keyword(clean_corpus):
    new_clean_corpus=[]
    merge_word_list=[] # n-gram keywords
    hit_row_index=[] # keep for predict test
    for no, line in enumerate(clean_corpus):
        words = line.split(' ')
        #words = word_tokenize(line) 
        words_len = len(words)
        # n-gram
        new_line=''
        for i in range(4, 1, -1):
            for j in range(words_len-i+1):
                merge_word = check_special_term(words, j, i)
                if merge_word == '':
                    new_line+=' '+words[j]
                    if j==words_len-i:
                        for k in range(j+1, words_len):
                            new_line+=' '+words[k]
                    continue
                else:
                    merge_word_list.append(merge_word)
                    hit_row_index.append(no)
                    new_line+=' '+merge_word
                    j+=i
                    continue

            # line is too short, keep it as original
            if words_len-i<0:
                new_line=line    
            new_line = new_line.strip()
            words = new_line.split(' ')
            words_len = len(words)
            if i>2:
                new_line=''
        new_clean_corpus.append(new_line)
        #if new_line != line:
        #    print('org_line='+line+'\n')
        #    print('new_line='+new_line+'\n\n')
    return new_clean_corpus, merge_word_list, hit_row_index

new_clean_corpus, merge_word_list, hit_row_index = compare_corpus_with_keyword(clean_corpus)
hit_row_index=list(set(hit_row_index))        

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
BOW_vector = vectorizer.fit_transform(new_clean_corpus)
X = BOW_vector.toarray()

vocab = vectorizer.get_feature_names()

In [45]:
# vectorizer.vocabulary_

In [46]:
import numpy as np
import guidedlda

#X = guidedlda.datasets.load_data(guidedlda.datasets.NYT)
#vocab = guidedlda.datasets.load_vocab(guidedlda.datasets.NYT)
word2id = dict((v, idx) for idx, v in enumerate(vocab))

print(X.shape)


(34012, 4634)


In [47]:
type(X)

numpy.ndarray

In [48]:
X[:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [49]:
print(X.sum())

117323


In [50]:
len(vocab)

4634

In [51]:
vocab[:10]

['aa',
 'abi',
 'ability',
 'able',
 'abo',
 'abort',
 'abr',
 'absent',
 'absolute',
 'acad']

In [52]:
# Normal LDA without seeding
model = guidedlda.GuidedLDA(n_topics=30, n_iter=100, random_state=7, refresh=20)
model.fit(X)


INFO:guidedlda:n_documents: 34012
INFO:guidedlda:vocab_size: 4634
INFO:guidedlda:n_words: 117323
INFO:guidedlda:n_topics: 30
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -1321626
INFO:guidedlda:<20> log likelihood: -792335
INFO:guidedlda:<40> log likelihood: -739597
INFO:guidedlda:<60> log likelihood: -717139
INFO:guidedlda:<80> log likelihood: -705122
INFO:guidedlda:<99> log likelihood: -696469


## 顯示每個Topics前八個重要的單字

In [53]:
topic_word = model.topic_word_
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))


Topic 0: security inquiry product agent deploy business service install
Topic 1: uninstall tool agent request security wfb password removal
Topic 2: agent security uninstall installation issue offline password uninstallation
Topic 3: service agent protection install start mac installation unable
Topic 4: false positive file detection application detect virus ufe
Topic 5: issue performance high usage cpu application wfb quickbook
Topic 6: file exclusion program block quarantine application exclude add
Topic 7: space disk inquiry server request log folder invalid
Topic 8: activate license account code change clp password new
Topic 9: service request threat support corporate inquiry master patch
Topic 10: file encryption unauthorized infection ransomware detect malware false
Topic 11: license seat issue add count information inquiry console
Topic 12: scan inquiry schedule time product report configuration notification
Topic 13: license inquiry concern threat product renewal status account

## 加重特定單字在某些Topics的權重

In [58]:
# Guided LDA with seed topics.
seed_topic_list = [['exhange', 'exhange-server', 'server', 'email'],
                   ['excel', 'office', 'powerpoint', 'word'],
                   ['install', 'antivirus'],
                   ['update', 'upgrade ', 'antivirus', 'migrate'],
                   ['uninstall', 'antivirus'],
                   ['license', 'expire', 'renew', 'activate'],
                   ['security', 'agent'],
                   ['web', 'url', 'http', 'block'],
                   ['console'],
                   ['performance', 'slow'],
                   ['virus', 'scan', 'malware', 'detect', 'infection'],
                  ]

model = guidedlda.GuidedLDA(n_topics=30, n_iter=100, random_state=7, refresh=20)
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        if not word in word2id.keys():
            word2id[word] = t_id

        seed_topics[word2id[word]] = t_id
model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)


INFO:guidedlda:n_documents: 34012
INFO:guidedlda:vocab_size: 4634
INFO:guidedlda:n_words: 117323
INFO:guidedlda:n_topics: 30
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -1314406
INFO:guidedlda:<20> log likelihood: -771597
INFO:guidedlda:<40> log likelihood: -725629
INFO:guidedlda:<60> log likelihood: -705498
INFO:guidedlda:<80> log likelihood: -696754
INFO:guidedlda:<99> log likelihood: -691446


## 顯示每個Topics前八個重要的單字

In [59]:
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))


Topic 0: server new migrate agent migration security wfb install client another
Topic 1: file false positive quarantine detection restore delete detect ufe unable
Topic 2: install agent unable installation error security fail mac patch remnant
Topic 3: update scan agent smart pattern service server security disconnected available
Topic 4: uninstall tool request agent password wfb security removal needs need
Topic 5: device file inquiry exclusion control analysis log folder virus issue
Topic 6: license seat activate issue renewal inquiry code account add renew
Topic 7: agent security install uninstall unable deploy offline installation reinstall assistance
Topic 8: url block filter website application program working reclassification whitelist approve
Topic 9: console agent web access showing offline unable page display appear
Topic 10: issue performance high usage cpu disk server space slow cause
Topic 11: scan schedule infection inquiry time malware smart possible update machine
Topic

## 顯示每個Document所屬的Topics

In [60]:
doc_topic = model.transform(X)
for i in range(19):
    doc=[]
    vocab_len = len(vocab)
    for j in range(vocab_len):
        if X[i, j] != 0:
            doc.append(vocab[j])
    print(' '.join(doc))
    
    print("top topic: {} Document: {}".format(doc_topic[i].argmax(),
          ', '.join(np.array(vocab)[list(reversed(X[i,:].argsort()))[0:5]])))
    print(' ')

feature full installation mac
top topic: 15 Document: mac, feature, full, installation, 顯示離線
 
scan smart update
top topic: 3 Document: smart, update, scan, hive, hkey
 
installation issue
top topic: 21 Document: issue, installation, hook, hlt, hoax
 
ransomware wallet
top topic: 15 Document: ransomware, wallet, 顯示離線, hlog, hlt
 
client detection machine possible ransomware wfb
top topic: 11 Document: machine, client, wfb, detection, possible
 
add behavior exclusion monitor
top topic: 16 Document: add, exclusion, monitor, behavior, home
 
migrate new old server
top topic: 0 Document: server, old, migrate, new, hold
 
agent clone
top topic: 7 Document: agent, clone, hook, hlt, hoax
 
agent function main micro security software
top topic: 7 Document: main, function, micro, agent, security
 
application detect false positive trojan
top topic: 25 Document: positive, false, detect, trojan, application
 
active consume disk folder large space update
top topic: 10 Document: update, active, d

## lighten the model object

In [61]:
# This step will delete some matrices inside the model.
model.purge_extra_matrices()

## Save the model 

In [62]:
from six.moves import cPickle as pickle
with open('guidedlda_model.pickle', 'wb') as file_handle:
    pickle.dump(model, file_handle)

## Load the model 

In [63]:
with open('guidedlda_model.pickle', 'rb') as file_handle:
    model = pickle.load(file_handle)

## re-test

In [64]:
doc_topic = model.transform(X)
for i in range(5):
    doc=[]
    vocab_len = len(vocab)
    for j in range(vocab_len):
        if X[i, j] != 0:
            doc.append(vocab[j])
    print(' '.join(doc))
    
    print("top topic: {} Document: {}".format(doc_topic[i].argmax(),
          ', '.join(np.array(vocab)[list(reversed(X[i,:].argsort()))[0:5]])))
    print(' ')

feature full installation mac
top topic: 15 Document: mac, feature, full, installation, 顯示離線
 
scan smart update
top topic: 3 Document: smart, update, scan, hive, hkey
 
installation issue
top topic: 21 Document: issue, installation, hook, hlt, hoax
 
ransomware wallet
top topic: 15 Document: ransomware, wallet, 顯示離線, hlog, hlt
 
client detection machine possible ransomware wfb
top topic: 11 Document: machine, client, wfb, detection, possible
 
